In [ ]:
!pip install liana

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.6/220.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [ ]:
import liana.method as method

In [ ]:
import pandas as pd
import os

 # Replace with the correct module from your list
import liana.resource as rs
import liana
import re
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import os

# # Specify the root directory
# root_dir = '/content/drive/My Drive'

# # Traverse and print all paths
# for dirpath, dirnames, filenames in os.walk(root_dir):
#     for dirname in dirnames:
#         print(os.path.join(dirpath, dirname))
#     for filename in filenames:
#         print(os.path.join(dirpath, filename))

In [ ]:
eshika, hratch = True, False
if eshika:
  data_path = '/content/drive/My Drive/Veritas AI'
if hratch:
  data_path = '/content/drive/My Drive/05_Other/VeritasAI/Eshika_Hratch/'

In [ ]:
# import os

# # Specify the root directory (you can change this to your specific folder if you know it)
# root_dir = '/content/drive/My Drive/Veritas AI'

# # Traverse and print all paths
# for dirpath, dirnames, filenames in os.walk(root_dir):
#     for dirname in dirnames:
#         print(os.path.join(dirpath, dirname))
#     for filename in filenames:
#         print(os.path.join(dirpath, filename))

In [ ]:
# Step 1: Select resources for cell-cell communication
ccc_resources = liana.rs.select_resource("consensus")
ccc_resources.head()

,ligand,receptor
0,LGALS9,PTPRC
1,LGALS9,MET
2,LGALS9,CD44
3,LGALS9,LRP1
4,LGALS9,CD47


In [ ]:
combined_column = pd.concat([ccc_resources['ligand'], ccc_resources['receptor']], ignore_index=True).drop_duplicates()
print('There are {} unique genes in the LIANA consensus ligand-receptor database'.format(combined_column.shape[0]))
combined_column.head()

There are 2017 unique genes in the LIANA consensus ligand-receptor database


,0
0,LGALS9
9,DLL1
13,PODXL
14,NRG2
16,ADGRB1


In [ ]:
test_data=combined_column.head(10)
test_data

,0
0,LGALS9
9,DLL1
13,PODXL
14,NRG2
16,ADGRB1
17,ADAM10
30,TNFSF11
33,EREG
36,ADAM12
38,TNFSF12


In [ ]:
pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 47.0 MB/s eta 0:00:00


In [ ]:
combined_column2= combined_column

In [ ]:
from Bio import Entrez
import time

# Set your email for NCBI queries
Entrez.email = "fairspringgal@gmail.com"

def fetch_refseq(combined_column):
    refseq_ids = []

    for protein in combined_column:
        print(f"Searching for: {protein}")  # Debugging

        # Search NCBI Gene database
        try:
            term = f"{protein}[Gene]"
            handle = Entrez.esearch(db="gene", term=term, retmode="xml")
            records = Entrez.read(handle)
            handle.close()
        except Exception as e:
            print(f"Error in esearch for {protein}: {e}")
            refseq_ids.append(f"Error fetching {protein}")
            continue

        if not records.get("IdList"):
            print(f"No Gene ID found for {protein}")
            refseq_ids.append(f"No RefSeq found for {protein}")
            continue

        gene_id = records["IdList"][0]  # Take first gene ID

        # Fetch the gene summary
        try:
            handle = Entrez.efetch(db="gene", id=gene_id, retmode="xml")
            gene_data = Entrez.read(handle)
            handle.close()
        except Exception as e:
            print(f"Error in efetch for {protein}: {e}")
            refseq_ids.append(f"Error fetching {protein}")
            continue

        # Extract NM_ RefSeq from gene summary
        try:
            refseq = gene_data[0]['Entrezgene_locus'][0]['Gene-commentary_products'][0]['Gene-commentary_accession']
            refseq_ids.append(refseq)
        except (KeyError, IndexError):
            print(f"No RefSeq found for {protein}")
            refseq_ids.append(f"No RefSeq found for {protein}")

        time.sleep(1)  # Prevent rate-limiting

    return refseq_ids

# Example usage
# combined_column should be defined elsewhere in your code
# combined_column = [...] # Your list of protein variants

# Fetch NM_ RefSeq IDs
refseq_ids = fetch_refseq(combined_column2)

# Print the results
for protein, refseq_id in zip(combined_column2, refseq_ids):
    dict_molecular_names = dict(zip(combined_column2, refseq_ids))
    print(f"{protein}: {refseq_id}")


Searching for: LGALS9
Searching for: DLL1
Searching for: PODXL
Searching for: NRG2
Searching for: ADGRB1
Searching for: ADAM10
Searching for: TNFSF11
Searching for: EREG
Searching for: ADAM12
Searching for: TNFSF12
Searching for: TNFSF14
Searching for: ADAM23
Searching for: ADAM11
Searching for: ICOSLG
Searching for: SEMA7A
Searching for: TNFSF13
Searching for: NFASC
Searching for: SLITRK3
Searching for: SLITRK5
Searching for: TNFSF15
Searching for: TGFA
Searching for: TNF
Searching for: HLA-B
Searching for: HLA-DRA
Searching for: HLA-DQA2
Searching for: HLA-DQA1
Searching for: HLA-DRB1
Searching for: HLA-DQB1
Searching for: ANXA1
Searching for: THY1
Searching for: HLA-A
Searching for: HLA-DPB1
Searching for: APP
Searching for: ITGB2
Searching for: ICAM1
Searching for: LCK
Searching for: FCER2
Searching for: ITGAV
Searching for: ANXA2
Searching for: GP1BA
Searching for: THBS1
Searching for: CD55
Searching for: CD14
Searching for: MELTF
Searching for: CD48
Searching for: CSF1
Searching 

In [ ]:
dict_molecular_names

{'LGALS9': 'NR_024043',
 'DLL1': 'XM_036160292',
 'PODXL': 'NM_005397',
 'NRG2': 'NM_001184935',
 'ADGRB1': 'XM_017013695',
 'ADAM10': 'NM_001320570',
 'TNFSF11': 'NM_033012',
 'EREG': 'NM_001432',
 'ADAM12': 'NM_001288973',
 'TNFSF12': 'NR_037146',
 'TNFSF14': 'NM_003807',
 'ADAM23': 'NM_001410985',
 'ADAM11': 'XR_388328',
 'ICOSLG': 'XM_036155889',
 'SEMA7A': 'NM_001146029',
 'TNFSF13': 'NR_073490',
 'NFASC': 'NM_001169234',
 'SLITRK3': 'NM_001318811',
 'SLITRK5': 'NM_001384609',
 'TNFSF15': 'NM_001204344',
 'TGFA': 'NM_001099691',
 'TNF': 'NM_000594',
 'HLA-B': 'NM_005514',
 'HLA-DRA': 'NM_019111',
 'HLA-DQA2': 'NM_020056',
 'HLA-DQA1': 'XM_006715079',
 'HLA-DRB1': 'NM_002124',
 'HLA-DQB1': 'NM_002123',
 'ANXA1': 'XM_017014657',
 'THY1': 'NM_009382',
 'HLA-A': 'NM_002116',
 'HLA-DPB1': 'NM_002121',
 'APP': 'NM_001136131',
 'ITGB2': 'NM_001127491',
 'ICAM1': 'NM_000201',
 'LCK': 'XM_024447046',
 'FCER2': 'XM_005272462',
 'ITGAV': 'NM_002210',
 'ANXA2': 'NM_004039',
 'GP1BA': 'NM_0001

In [ ]:
from Bio import SeqIO

In [ ]:
values = dict_molecular_names.keys()
print(list(values)) # Output: [1, 2, 3]

['LGALS9', 'DLL1', 'PODXL', 'NRG2', 'ADGRB1', 'ADAM10', 'TNFSF11', 'EREG', 'ADAM12', 'TNFSF12', 'TNFSF14', 'ADAM23', 'ADAM11', 'ICOSLG', 'SEMA7A', 'TNFSF13', 'NFASC', 'SLITRK3', 'SLITRK5', 'TNFSF15', 'TGFA', 'TNF', 'HLA-B', 'HLA-DRA', 'HLA-DQA2', 'HLA-DQA1', 'HLA-DRB1', 'HLA-DQB1', 'ANXA1', 'THY1', 'HLA-A', 'HLA-DPB1', 'APP', 'ITGB2', 'ICAM1', 'LCK', 'FCER2', 'ITGAV', 'ANXA2', 'GP1BA', 'THBS1', 'CD55', 'CD14', 'MELTF', 'CD48', 'CSF1', 'HLA-C', 'TGFB3', 'CD28', 'ACE', 'CDH1', 'TDGF1', 'NCAM1', 'ICAM2', 'CEACAM1', 'HLA-E', 'HLA-DRB4', 'HLA-DOB', 'SELL', 'CD99', 'PLA2G2A', 'CD1D', 'ST6GAL1', 'SPN', 'SELE', 'HLA-G', 'CD58', 'VCAM1', 'HLA-DPA1', 'CD22', 'EFNA1', 'KITLG', 'MRC1', 'MPZ', 'HLA-DMA', 'HLA-DMB', 'CD34', 'CD40LG', 'HLA-F', 'CEACAM8', 'L1CAM', 'ICAM3', 'CD80', 'SDC2', 'BTC', 'CEACAM6', 'CD96', 'CD200', 'TNFSF9', 'CD86', 'GDF5', 'BAG6', 'FASLG', 'FLT3LG', 'TNFSF10', 'EFNA4', 'EFNB2', 'EFNA5', 'NRG3', 'JAM2', 'NRXN1', 'NRXN2', 'B2M', 'CXADR', 'SIRPA', 'JAG1', 'ADAM17', 'HLA-DRB3', '

In [ ]:
from Bio import Entrez, SeqIO

# Set email for Entrez
Entrez.email = "fairspringgal@gmail.com"

# Function to fetch protein sequence
def fetch_protein_sequence(mrna_id):
    try:
        handle = Entrez.efetch(db="nucleotide", id=mrna_id, rettype="gb", retmode="text")
        record = SeqIO.read(handle, "genbank")
        handle.close()

        for feature in record.features:
            if feature.type == "CDS" and "translation" in feature.qualifiers:
                return feature.qualifiers["translation"][0]
        return "Translation not available"
    except Exception as e:
        return f"Error fetching data for {mrna_id}: {e}"

# Dictionary to store protein sequences
liana_sequence = {}

# Ensure refseq_ids matches the molecular names dictionary
values = list(dict_molecular_names.keys())

for protein in refseq_ids:
    protein_seq = fetch_protein_sequence(protein) or "NAN"  # Fix: Default to 'NAN' if None
    index = refseq_ids.index(protein)  # Get corresponding name
    liana_sequence[values[index]] = protein_seq  # Store in dictionary

# Print results
print(liana_sequence)
print(f"Number of sequences retrieved: {len(liana_sequence)}")


{'LGALS9': 'Translation not available', 'DLL1': 'MGRRSALALAVVSALLCQVWSSGVFELKLQEFVNKKGLLGNRNCCRGGSGPPCACRTFFRVCLKHYQASVSPEPPCTYGSAVTPVLGVDSFSLPDGAGIDPAFSNPIRFPFGFTWPGTFSLIIEALHTDSPDDLATENPERLISRLTTQRHLTVGEEWSQDLHSSGRTDLRYSYRFVCDEHYYGEGCSVFCRPRDDAFGHFTCGDRGEKMCDPGWKGQYCTDPICLPGCDDQHGYCDKPGECKCRVGWQGRYCDECIRYPGCLHGTCQQPWQCNCQEGWGGLFCNQDLNYCTHHKPCRNGATCTNTGQGSYTCSCRPGYTGANCELEVDECAPSPCKNGASCTDLEDSFSCTCPPGFYGKVCELSAMTCADGPCFNGGRCSDNPDGGYTCHCPLGFSGFNCEKKMDLCGSSPCSNGAKCVDLGNSYLCRCQAGFSGRYCEDNVDDCASSPCANGGTCRDSVNDFSCTCPPGYTGKNCSAPVSRCEHAPCHNGATCHQRGQRYMCECAQGYGGPNCQFLLPEPPPGPMVVDLSERHMESQGGPFPWVAVCAGVVLVLLLLLGCAAVVVCVRLKLQKHQPPPEPCGGETETMNNLANCQREKDVSVSIIGATQIKNTNKKADFHGDHGAEKSSFKVRYPTVDYNLVRDLKGDEATVRDTHSKRDTKCQSQSSAGEEKIAPTLRGGEIPDRKRPESVYSTSKDTKYQSVYVLSAEKDECVIATEVSVRPSCRGRE', 'PODXL': 'MRCALALSALLLLLSTPPLLPSSPSPSPSPSQNATQTTTDSSNKTAPTPASSVTIMATDTAQQSTVPTSKANEILASVKATTLGVSSDSPGTTTLAQQVSGPVNTTVARGGGSGNPTTTIESPKSTKSADTTTVATSTATAKPNTTSSQNGAEDTTNSGGKSSHSVTTDLTSTKAEHLTTPHPTSPLSPRQPTSTHPVATPTSSGH

In [ ]:
my_dict = liana_sequence
value_to_remove = 'Translation not available'
for key, value in list(my_dict.items()): #Iterate over a copy
    if value == value_to_remove:
        del my_dict[key]
print(len(my_dict))

1710


In [ ]:
liana_sequence = list(my_dict.values())
liana_sequence

['...',
 'Error fetching data for Error fetching B2M: HTTP Error 400: Bad Request',
 '...',
 'Error fetching data for No RefSeq found for FRAS1_FREM1_NPNT: HTTP Error 400: Bad Request',
 'Error fetching data for No RefSeq found for FRAS1_FREM2_NPNT: HTTP Error 400: Bad Request',
 '...',
 'Error fetching data for No RefSeq found for CLCF1_CRLF1: HTTP Error 400: Bad Request',
 'Error fetching data for No RefSeq found for LTA_LTB: HTTP Error 400: Bad Request',
 'Error fetching data for No RefSeq found for IGHG1_IGLC1: HTTP Error 400: Bad Request',
 'Error fetching data for No RefSeq found for INHA_INHBA: HTTP Error 400: Bad Request',
 'Error fetching data for No RefSeq found for INHA_INHBB: HTTP Error 400: Bad Request',
 'Error fetching data for No RefSeq found for INHBA_INHBB: HTTP Error 400: Bad Request',
 'Error fetching data for No RefSeq found for CSF1_IL34: HTTP Error 400: Bad Request',
 'Error fetching data for No RefSeq found for IL12A_IL12B: HTTP Error 400: Bad Request',
 'Error 

In [ ]:
# from Bio.SeqUtils.ProtParam import ProteinAnalysis
# X = ProteinAnalysis("MAEGEITTFTALTEKFNLPPGNYKKPKLLYCSNGGHFLRILPDGTVDGT"
#                     "RDRSDQHIQLQLSAESVGEVYIKSTETGQYLAMDTSGLLYGSQTPSEEC"
#                     "LFLERLEENHYNTYTSKKHAEKNWFVGLKKNGSCKRGPRTHYGQKAILF"
#                     "LPLPV")
# print("%0.2f" % X.molecular_weight())

In [ ]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis

valid_amino_acids = set("ACDEFGHIKLMNPQRSTVWY")  # Standard 20 amino acids
molecular_weights = {}

liana_sequence = list(my_dict.values())  # List of sequences
liana_names = list(my_dict.keys())      # List of names (keys)

for index, sequence in enumerate(liana_sequence):
    # Optional: Check for invalid sequences
    if not set(sequence).issubset(valid_amino_acids):
        print(f"Skipping invalid sequence: {sequence}")
        molecular_weights[liana_names[index]] = 'NAN'  # Assign None for invalid sequences
        # continue

    prot_analysis = ProteinAnalysis(sequence)
    molecular_weight_pre = prot_analysis.molecular_weight()  # Compute molecular weight

    # Store molecular weight in dictionary using liana_names[index] as key
    molecular_weights[liana_names[index]] = molecular_weight_pre

print("Molecular weights calculated successfully.")
print(molecular_weights)  # Check the output


Skipping invalid sequence: Error fetching data for Error fetching B2M: HTTP Error 400: Bad Request


ValueError: ''B'' is not a valid unambiguous letter for protein

In [ ]:
liana_gene_name = list(molecular_weights.keys())

In [ ]:
molecular_weights_liana = list(molecular_weights.values())

In [ ]:
 df2 = pd.DataFrame({'liana_gene_name': liana_gene_name, 'molecular_weights': molecular_weights_liana })

In [ ]:
df2.to_csv('/content/drive/My Drive/Veritas AI/getmolecularweights_data.csv', index=False)